In [1]:
from interactive_visualization.animation_utils import animate_list
from interactive_visualization.graph_utils.graph import Graph, Arc, Node

In [35]:
def state(arcs, flows, current_augmenting_path = None, s_cut = None):
    arcs_visualized = [Arc(u, v, f"{f}/{c}") for (u, v, c), f in zip(arcs, flows)]
    
    if current_augmenting_path is not None:
        for arc_num in current_augmenting_path:
            arcs_visualized[arc_num].SetColor('red')
            
    graph = Graph(arcs_visualized)
    if s_cut is not None:
        for node_id in s_cut:
            graph.nodes[node_id].SetColor('blue')

    if current_augmenting_path is not None:
        for arc_num in current_augmenting_path:
            graph.nodes[arcs[arc_num][0]].SetColor('red')
            graph.nodes[arcs[arc_num][1]].SetColor('red')
            
            
    return graph.Visualize()

In [39]:
def ford_fulkerson(arcs, s, t):
    states = []
    n = max(max([arc[0] for arc in arcs]), max([arc[1] for arc in arcs])) + 1
    arcs_adjacent = [[] for _ in range(n)]
    for i, arc in enumerate(arcs):
        arcs_adjacent[arc[0]].append(i)
        arcs_adjacent[arc[1]].append(i)
        
    flows = [0 for _ in arcs]
        
    # DFS-like augmenting path procedure, it finds a path we can send flow along
    # and the return value is the maximum possible flow amount to be send through this path
    def find_augmenting_path(v, arcs_adjacent, arcs, flows, t, flow, visited, path):
        if v == t:
            return flow
        if v in visited:
            return 0
        visited.add(v)
        
        for arc_num in arcs_adjacent[v]:
            arc = arcs[arc_num]
            new_flow = 0
            if v == arc[0] and flows[arc_num] < arc[2]:
                new_flow = find_augmenting_path(arc[1], arcs_adjacent, arcs, flows, t, min(flow, arc[2] - flows[arc_num]), visited, path)
            elif v == arc[1] and flows[arc_num] > 0:
                new_flow = find_augmenting_path(arc[0], arcs_adjacent, arcs, flows, t, min(flow, flows[arc_num]), visited, path)
            if new_flow > 0:
                path.append((arc_num, 1 if v == arc[0] else -1))
                return new_flow
        return 0
    
    max_cap = max([arc[2] for arc in arcs])
    
    visited = set()
    path = []
    # Here's the main loop, mainly while we find an augmenting path, augment flow through it
    # with the maximum possible amount and continue
    flow = find_augmenting_path(s, arcs_adjacent, arcs, flows, t, max_cap, visited, path)
    states.append(state(arcs, flows))
    while flow > 0:
        for arc_num, direction in path:
            flows[arc_num] += direction * flow
        path = [arc_num for arc_num, _ in path]
        states.append(state(arcs, flows, path, None))
        visited = set()
        path = []
        flow = find_augmenting_path(s, arcs_adjacent, arcs, flows, t, max_cap, visited, path)
        states.append(state(arcs, flows))
        
    states.append(state(arcs, flows, None, visited))
    return animate_list(states, play=True, interval=1000)

In [40]:
arcs = [
    (0, 1, 7),
    (0, 2, 4),
    (1, 2, 4),
    (1, 3, 2),
    (2, 3, 8),
    (2, 4, 4),
    (3, 4, 4),
    (3, 5, 5),
    (4, 5, 12)
]
states = ford_fulkerson(arcs, 0, 5)

interactive(children=(IntSlider(value=0, description='step', max=7), Output()), _dom_classes=('widget-interact…